In [48]:
from gliner import GLiNER   # model
import re                   # data cleaning
import pandas as pd         # data handling

In [49]:
model = GLiNER.from_pretrained("urchade/gliner_multi")

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 51463.85it/s]
/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [50]:
# test model (text from one of the news)
text = """
"CAMERON HIGHLANDS: Peniaga di Cameron Highlands bimbang jika sekatan merentas daerah atau negeri dilaksanakan semula berikutan kes Covid-19 yang semakin meningkat kerana ia akan menjejaskan perniagaan mereka.

Peniaga khuatir perniagaan yang baharu hendak pulih akan kembali terjejas teruk seperti semasa Perintah Kawalan Pergerakan (PKP) dilaksanakan sebelum ini.

Pemilik perniagaan Siqkliz Enterprise yang menjual strawberi, pakaian terpakai dan cenderamata, Mohd. Kazz Zakaria, 39, berkata, walaupun sudah beberapa bulan kembali beroperasi seperti biasa, perniagaannya masih belum mampu menampung kerugian semasa tempoh PKP.

Menurutnya, sewaktu tempoh PKP, Cameron Highlands lengang, menyebabkan perniagaan tidak dapat dijalankan.

“Sewaktu tempoh PKP, memang terduduk jika fikirkan perniagaan. Saya terpaksa mencari alternatif lain bagi memastikan buah strawberi boleh dijual. Kejadian tanah runtuh pada Disember tahun lalu menambahkan lagi kegusaran pengunjung untuk melancong ke sini.

“Selepas kejadian tanah runtuh itu, jumlah pengunjung semakin kurang malah hujung minggu pun macam tiada orang,” katanya ketika ditemui Utusan Malaysia.

Mohd. Kazz berharap kerajaan tidak melaksanakan sekatan disebabkan kes Covid-19 yang meningkat yang akan menjejaskan perniagaannya.

“Saya harap kerajaan tidak membuat sekatan yang boleh menjejaskan perniagaan kerana sewaktu tempoh PKP, kami terpaksa menjualnya ke Lembah Klang tetapi strawberi hanya mampu bertahan selama tiga hari, jika tidak kualitinya akan terjejas,” katanya.

R. Sentharamai, 49, peniaga di Tanah Rata berkata, dia lega apabila dapat berniaga semula namun tetap bimbang jika pergerakan rentas daerah dan negeri tidak dibenarkan.

“Perniagaan kami bergantung sepenuhnya kepada agro dan pelancongan. Sewaktu tempoh PKP, perniagaan tutup, kini hanya buka Jumaat, Sabtu dan Ahad tetapi sekarang pun pelanggan tak ramai,” katanya.

Tantrisyalini, 25, yang bekerja di Kedai Cenderamata Adik Aisyah berkata, gajinya tetap dibayar oleh majikannya yang menjalankan perniagaan secara dalam talian.

“Risau juga jika PKP kembali kerana kami sudah mula beroperasi seperti biasa, banyak kos yang perlu ditanggung,” katanya."
"""

labels = ["GPE","PERSON","ORG","FAC","MONEY","NORP","LOC","PRODUCT","EVENT",
          "PERCENT","WORK_OF_ART","TIME","ORDINAL","CARDINAL","QUANTITY","LAW"]

In [51]:
# split text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

In [52]:
# process individual sentences
all_entities = []
current_position = 0

for sentence in sentences:
    if not sentence.strip():
        current_position += len(sentence) + 1
        continue
        
    # get predictions
    entities = model.predict_entities(sentence, labels)
    
    # adjust position
    for entity in entities:
        entity["start"] += current_position
        entity["end"] += current_position
        all_entities.append(entity)
    
    # update position
    current_position += len(sentence) + 1

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [53]:
# Remove duplicates while preserving order
seen = set()
unique_entities = []
for entity in all_entities:
    identifier = (entity["text"], entity["label"], entity["start"], entity["end"])
    if identifier not in seen:
        seen.add(identifier)
        unique_entities.append(entity)

In [54]:
# results
for entity in unique_entities:
    print(f"{entity['text']} => {entity['label']}")

CAMERON HIGHLANDS => LOC
Peniaga => PERSON
Cameron Highlands => LOC
negeri => LOC
Covid-19 => LAW
perniagaan => PRODUCT
Peniaga => PERSON
perniagaan => ORG
Perintah Kawalan Pergerakan => LAW
Siqkliz Enterprise => ORG
strawberi => PRODUCT
pakaian terpakai => PRODUCT
cenderamata => PRODUCT
Mohd. => PERSON
Kazz Zakaria => PERSON
beberapa bulan => TIME
perniagaannya => ORG
PKP => ORG
PKP => EVENT
Cameron Highlands => LOC
perniagaan => ORG
PKP => ORG
perniagaan => ORG
Saya => PERSON
buah strawberi => PRODUCT
tanah runtuh => EVENT
Disember => TIME
pengunjung => PERSON
sini => LOC
tanah runtuh => EVENT
pengunjung => PERSON
hujung minggu => TIME
Utusan Malaysia => ORG
Mohd. => PERSON
Kazz => PERSON
kerajaan => ORG
sekatan => LAW
Covid-19 => EVENT
perniagaannya => PRODUCT
kerajaan => ORG
perniagaan => PRODUCT
PKP => EVENT
Lembah Klang => LOC
strawberi => PRODUCT
tiga hari => TIME
R. => ORG
Sentharamai => PERSON
49 => QUANTITY
Tanah Rata => LOC
negeri => LOC
Perniagaan => ORG
agro => FAC
pelanco

In [55]:
# convert to df
df = pd.DataFrame(unique_entities)
df.head()

,start,end,text,label,score
0,2,19,CAMERON HIGHLANDS,LOC,0.956421
1,21,28,Peniaga,PERSON,0.837058
2,32,49,Cameron Highlands,LOC,0.979273
3,92,98,negeri,LOC,0.762886
4,133,141,Covid-19,LAW,0.692249


In [56]:
# save as csv
df.to_csv("results_test.csv", index=False)